# Correlation 

**Correlation** 
* A meausre of the statistical relationship of 2 or more variable 
* Correlation predictor variables provide redundant information 
* Variables should be correlated with the target but not among themselves 


**Correlation and ML** 

* Correlation features do not necessarily affect model accuracy per se
* High **dimensionality** does 
* If 2 features are highly correlated, the second one will add little information: removing it helps reduce dimension 
* Correlation affects model **interpretability**: linear models 
* Different classifier show different sensitivityt to correlation


**Types** 

* Pearson's correlation coefficient (linear relationship)
* Spearman's rank correlation coefficient 
* Kendall rank correlation coefficient


**Pearson's correlation** 

$$
  \frac{S_{xy}}{S_xS_y}=\frac{\sum{(x_i-\bar{x})(y_i-\bar{y})}}{\sqrt{\sum{(x_i-\bar{x})^2}\sum{(y_i-\bar{y})^2}}}
  $$
 
 
**Correlated variable removal methods**
* Method1: Brute force method
  * Scan features as they appear. If a feature is correlated, remove the correlated feature
  * Pro: Fast 
  * Cons: We may remove the feature more important than the other if it appears later
* Method 2: 
  * Steps
    * Identify groups of correlated features 
    * Select the most predictive feature
      * Build a small machine learning model using the features in the group 
      * Other criteria, e.g. number of missing values 
    * Discard the rest 